# 여러 시도를 해보는 코드!

In [2]:
import torch

a= torch.dot(torch.tensor([2,3]),torch.tensor([2,1]))
torch.bmm(torch.FloatTensor(10,3,5),torch.FloatTensor(10,5,3)).size()

torch.Size([10, 3, 3])

In [3]:
import numpy as np
data=np.load("../data/cats/fid_files/cats_64.npz")

In [4]:
data["m"].shape

(2048,)

In [5]:
import torch
import math
import numpy as np
# num_step = 4, 배치 = 3, 크기는 = 8*8
def normalize_vecs(vectors: torch.Tensor) -> torch.Tensor:
    """
    Normalize vector lengths.
    """
    return vectors / (torch.norm(vectors, dim=-1, keepdim=True))

x, y = torch.meshgrid(torch.linspace(-1, 1, 8, ),
                          torch.linspace(-1, 1, 8))
x=x.T.flatten()
y=y.T.flatten()
z = -torch.ones_like(x) / np.tan((2 * math.pi * 30 / 360)/2)

ray_d_cam= normalize_vecs(torch.stack([x, y, z], -1))
z_vals=torch.linspace(2, 6, 4).reshape(1, 4, 1).repeat(64, 1, 1)
points = ray_d_cam.unsqueeze(1).repeat(1, 4, 1) * z_vals
points = torch.stack(3*[points]) 
z_vals = torch.stack(3*[z_vals]) 
ray_d_cam = torch.stack(3*[ray_d_cam]) 

/opt/anaconda3/envs/jaepoong/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1656313733050/work/aten/src/ATen/native/TensorShape.cpp:2890.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
def transform_sampled_points(points, z_vals, ray_directions, h_stddev=1, v_stddev=1, h_mean=math.pi * 0.5, v_mean=math.pi * 0.5, mode='normal'):
    """Samples a camera position and maps points in camera space to world space."""

    n, num_rays, num_steps, channels = points.shape

    points, z_vals = perturb_points(points, z_vals, ray_directions)


    camera_origin, pitch, yaw = sample_camera_positions(n=points.shape[0], r=1, horizontal_stddev=h_stddev, vertical_stddev=v_stddev, horizontal_mean=h_mean, vertical_mean=v_mean, mode=mode)
    forward_vector = normalize_vecs(-camera_origin)

    cam2world_matrix = create_cam2world_matrix(forward_vector, camera_origin)

    points_homogeneous = torch.ones((points.shape[0], points.shape[1], points.shape[2], points.shape[3] + 1))
    points_homogeneous[:, :, :, :3] = points

    # should be n x 4 x 4 , n x r^2 x num_steps x 4
    transformed_points = torch.bmm(cam2world_matrix, points_homogeneous.reshape(n, -1, 4).permute(0,2,1)).permute(0, 2, 1).reshape(n, num_rays, num_steps, 4)


    transformed_ray_directions = torch.bmm(cam2world_matrix[..., :3, :3], ray_directions.reshape(n, -1, 3).permute(0,2,1)).permute(0, 2, 1).reshape(n, num_rays, 3)

    homogeneous_origins = torch.zeros((n, 4, num_rays))
    homogeneous_origins[:, 3, :] = 1
    transformed_ray_origins = torch.bmm(cam2world_matrix, homogeneous_origins).permute(0, 2, 1).reshape(n, num_rays, 4)[..., :3]

    return transformed_points[..., :3], z_vals, transformed_ray_directions, transformed_ray_origins, pitch, yaw


In [7]:
def perturb_points(points, z_vals, ray_directions):
    distance_between_points = z_vals[:,:,1:2,:] - z_vals[:,:,0:1,:]
    offset = (torch.rand(z_vals.shape)-0.5) * distance_between_points
    z_vals = z_vals + offset

    points = points + offset * ray_directions.unsqueeze(2)
    return points, z_vals

perturb_points(points,z_vals,ray_d_cam)

(tensor([[[[-0.3530, -0.3530, -1.3174],
           [-0.8112, -0.8112, -3.0273],
           [-1.0648, -1.0648, -3.9738],
           [-1.4501, -1.4501, -5.4117]],
 
          [[-0.4066, -0.5692, -2.1243],
           [-0.5158, -0.7221, -2.6948],
           [-0.8775, -1.2285, -4.5849],
           [-1.0179, -1.4250, -5.3182]],
 
          [[-0.1594, -0.3720, -1.3883],
           [-0.3521, -0.8216, -3.0663],
           [-0.5299, -1.2365, -4.6145],
           [-0.7279, -1.6984, -6.3387]],
 
          ...,
 
          [[ 0.2583,  0.6027, -2.2491],
           [ 0.3559,  0.8305, -3.0994],
           [ 0.4536,  1.0584, -3.9500],
           [ 0.6342,  1.4797, -5.5224]],
 
          [[ 0.3474,  0.4863, -1.8150],
           [ 0.6536,  0.9150, -3.4147],
           [ 0.8252,  1.1553, -4.3116],
           [ 1.0296,  1.4414, -5.3795]],
 
          [[ 0.6286,  0.6286, -2.3460],
           [ 0.8522,  0.8522, -3.1806],
           [ 1.0245,  1.0245, -3.8236],
           [ 1.4180,  1.4180, -5.2922]]],
 
 
  

In [8]:
z_vals.shape

torch.Size([3, 64, 4, 1])

In [9]:
import math
import numpy as np
z=-torch.ones_like(x)/np.tan((2 * math.pi * 30 / 360)/2)

def normalize_vecs(vectors: torch.Tensor) -> torch.Tensor:
    """
    Normalize vector lengths.
    """
    return vectors / (torch.norm(vectors, dim=-1, keepdim=True))

normalize_vecs(torch.stack([x, y, z], -1))

tensor([[-0.2506, -0.2506, -0.9351],
        [-0.1818, -0.2545, -0.9498],
        [-0.1102, -0.2572, -0.9600],
        [-0.0369, -0.2586, -0.9653],
        [ 0.0369, -0.2586, -0.9653],
        [ 0.1102, -0.2572, -0.9600],
        [ 0.1818, -0.2545, -0.9498],
        [ 0.2506, -0.2506, -0.9351],
        [-0.2545, -0.1818, -0.9498],
        [-0.1847, -0.1847, -0.9653],
        [-0.1121, -0.1868, -0.9760],
        [-0.0376, -0.1878, -0.9815],
        [ 0.0376, -0.1878, -0.9815],
        [ 0.1121, -0.1868, -0.9760],
        [ 0.1847, -0.1847, -0.9653],
        [ 0.2545, -0.1818, -0.9498],
        [-0.2572, -0.1102, -0.9600],
        [-0.1868, -0.1121, -0.9760],
        [-0.1134, -0.1134, -0.9871],
        [-0.0380, -0.1140, -0.9928],
        [ 0.0380, -0.1140, -0.9928],
        [ 0.1134, -0.1134, -0.9871],
        [ 0.1868, -0.1121, -0.9760],
        [ 0.2572, -0.1102, -0.9600],
        [-0.2586, -0.0369, -0.9653],
        [-0.1878, -0.0376, -0.9815],
        [-0.1140, -0.0380, -0.9928],
 

In [10]:
torch.stack([x,y,z],-1)

tensor([[-1.0000, -1.0000, -3.7321],
        [-0.7143, -1.0000, -3.7321],
        [-0.4286, -1.0000, -3.7321],
        [-0.1429, -1.0000, -3.7321],
        [ 0.1429, -1.0000, -3.7321],
        [ 0.4286, -1.0000, -3.7321],
        [ 0.7143, -1.0000, -3.7321],
        [ 1.0000, -1.0000, -3.7321],
        [-1.0000, -0.7143, -3.7321],
        [-0.7143, -0.7143, -3.7321],
        [-0.4286, -0.7143, -3.7321],
        [-0.1429, -0.7143, -3.7321],
        [ 0.1429, -0.7143, -3.7321],
        [ 0.4286, -0.7143, -3.7321],
        [ 0.7143, -0.7143, -3.7321],
        [ 1.0000, -0.7143, -3.7321],
        [-1.0000, -0.4286, -3.7321],
        [-0.7143, -0.4286, -3.7321],
        [-0.4286, -0.4286, -3.7321],
        [-0.1429, -0.4286, -3.7321],
        [ 0.1429, -0.4286, -3.7321],
        [ 0.4286, -0.4286, -3.7321],
        [ 0.7143, -0.4286, -3.7321],
        [ 1.0000, -0.4286, -3.7321],
        [-1.0000, -0.1429, -3.7321],
        [-0.7143, -0.1429, -3.7321],
        [-0.4286, -0.1429, -3.7321],
 

In [11]:
def get_initial_rays_trig(n, num_steps, fov, resolution, ray_start, ray_end):
    """Returns sample points, z_vals, and ray directions in camera space."""

    W, H = resolution
    # Create full screen NDC (-1 to +1) coords [x, y, 0, 1].
    # Y is flipped to follow image memory layouts.
    x, y = torch.meshgrid(torch.linspace(-1, 1, W),
                          torch.linspace(1, -1, H, )) # x,y grid , it descrete -1 to 1 with W resolution
    x = x.T.flatten() # transform하면 적절하게 x좌표에 맞는다.
    y = y.T.flatten() #  transform하면 잘 맞는다.
    z = -torch.ones_like(x) / np.tan((2 * math.pi * fov / 360)/2) # 

    rays_d_cam = normalize_vecs(torch.stack([x, y, z], -1))


    z_vals = torch.linspace(ray_start, ray_end, num_steps).reshape(1, num_steps, 1).repeat(W*H, 1, 1)
    points = rays_d_cam.unsqueeze(1).repeat(1, num_steps, 1) * z_vals

    points = torch.stack(n*[points])
    z_vals = torch.stack(n*[z_vals])
    rays_d_cam = torch.stack(n*[rays_d_cam])

    return points, z_vals, rays_d_cam

points,z_vals,rays_d_cam=get_initial_rays_trig(3,4,30,(8,8),2,6)


In [12]:
points.shape,z_vals.shape,rays_d_cam.shape

(torch.Size([3, 64, 4, 3]), torch.Size([3, 64, 4, 1]), torch.Size([3, 64, 3]))

In [13]:
x, y = torch.meshgrid(torch.linspace(-1, 1, 8, ),
                          torch.linspace(-1, 1, 8))
x,y

(tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-0.7143, -0.7143, -0.7143, -0.7143, -0.7143, -0.7143, -0.7143, -0.7143],
         [-0.4286, -0.4286, -0.4286, -0.4286, -0.4286, -0.4286, -0.4286, -0.4286],
         [-0.1429, -0.1429, -0.1429, -0.1429, -0.1429, -0.1429, -0.1429, -0.1429],
         [ 0.1429,  0.1429,  0.1429,  0.1429,  0.1429,  0.1429,  0.1429,  0.1429],
         [ 0.4286,  0.4286,  0.4286,  0.4286,  0.4286,  0.4286,  0.4286,  0.4286],
         [ 0.7143,  0.7143,  0.7143,  0.7143,  0.7143,  0.7143,  0.7143,  0.7143],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000]]),
 tensor([[-1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000],
         [-1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000],
         [-1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000],
         [-1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000],
  

In [14]:
x.T.flatten()

tensor([-1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000,
        -1.0000, -0.7143, -0.4286, -0.1429,  0.1429,  0.4286,  0.7143,  1.0000])

In [15]:
z = -torch.ones_like(x) / np.tan((2 * math.pi * 30 / 360)/2)
z

tensor([[-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321],
        [-3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321, -3.7321]])

In [16]:
z_vals=torch.linspace(2, 6, 4).reshape(1, 4, 1).repeat(64, 1, 1)
z_vals.shape

torch.Size([64, 4, 1])

In [17]:
ray_d_cam= normalize_vecs(torch.stack([x, y, z], -1))
print(ray_d_cam.shape)
print(ray_d_cam)

torch.Size([8, 8, 3])
tensor([[[-0.2506, -0.2506, -0.9351],
         [-0.2545, -0.1818, -0.9498],
         [-0.2572, -0.1102, -0.9600],
         [-0.2586, -0.0369, -0.9653],
         [-0.2586,  0.0369, -0.9653],
         [-0.2572,  0.1102, -0.9600],
         [-0.2545,  0.1818, -0.9498],
         [-0.2506,  0.2506, -0.9351]],

        [[-0.1818, -0.2545, -0.9498],
         [-0.1847, -0.1847, -0.9653],
         [-0.1868, -0.1121, -0.9760],
         [-0.1878, -0.0376, -0.9815],
         [-0.1878,  0.0376, -0.9815],
         [-0.1868,  0.1121, -0.9760],
         [-0.1847,  0.1847, -0.9653],
         [-0.1818,  0.2545, -0.9498]],

        [[-0.1102, -0.2572, -0.9600],
         [-0.1121, -0.1868, -0.9760],
         [-0.1134, -0.1134, -0.9871],
         [-0.1140, -0.0380, -0.9928],
         [-0.1140,  0.0380, -0.9928],
         [-0.1134,  0.1134, -0.9871],
         [-0.1121,  0.1868, -0.9760],
         [-0.1102,  0.2572, -0.9600]],

        [[-0.0369, -0.2586, -0.9653],
         [-0.0376, -0.

In [29]:
def normalize_vecs(vectors: torch.Tensor) -> torch.Tensor:
    """
    Normalize vector lengths.
    """
    return vectors / (torch.norm(vectors, dim=-1, keepdim=True))

x, y = torch.meshgrid(torch.linspace(-1, 1, 8, ),
                          torch.linspace(-1, 1, 8))
x=x.T.flatten()
y=y.T.flatten()
z = -torch.ones_like(x) / np.tan((2 * math.pi * 30 / 360)/2)

ray_d_cam= normalize_vecs(torch.stack([x, y, z], -1))
z_vals=torch.linspace(2, 6, 4).reshape(1, 4, 1).repeat(64, 1, 1)
points = ray_d_cam.unsqueeze(1).repeat(1, 4, 1) * z_vals
print(points.shape)
print(points)

torch.Size([64, 4, 3])
tensor([[[-0.5011, -0.5011, -1.8702],
         [-0.8352, -0.8352, -3.1170],
         [-1.1693, -1.1693, -4.3639],
         [-1.5034, -1.5034, -5.6107]],

        [[-0.3636, -0.5090, -1.8997],
         [-0.6060, -0.8484, -3.1661],
         [-0.8484, -1.1877, -4.4325],
         [-1.0907, -1.5270, -5.6990]],

        [[-0.2205, -0.5145, -1.9201],
         [-0.3675, -0.8575, -3.2001],
         [-0.5145, -1.2005, -4.4802],
         [-0.6615, -1.5434, -5.7602]],

        [[-0.0739, -0.5173, -1.9305],
         [-0.1232, -0.8621, -3.2176],
         [-0.1724, -1.2070, -4.5046],
         [-0.2217, -1.5519, -5.7916]],

        [[ 0.0739, -0.5173, -1.9305],
         [ 0.1232, -0.8621, -3.2176],
         [ 0.1724, -1.2070, -4.5046],
         [ 0.2217, -1.5519, -5.7916]],

        [[ 0.2205, -0.5145, -1.9201],
         [ 0.3675, -0.8575, -3.2001],
         [ 0.5145, -1.2005, -4.4802],
         [ 0.6615, -1.5434, -5.7602]],

        [[ 0.3636, -0.5090, -1.8997],
         [ 0.60

In [19]:
import random
def sample_camera_positions( n=1, r=1, horizontal_stddev=1, vertical_stddev=1, horizontal_mean=math.pi*0.5, vertical_mean=math.pi*0.5, mode='normal'):
    """
    Samples n random locations along a sphere of radius r. Uses the specified distribution.
    Theta is yaw in radians (-pi, pi)
    Phi is pitch in radians (0, pi)
    """

    if mode == 'uniform':
        theta = (torch.rand((n, 1)) - 0.5) * 2 * horizontal_stddev + horizontal_mean
        phi = (torch.rand((n, 1)) - 0.5) * 2 * vertical_stddev + vertical_mean
				

    elif mode == 'normal' or mode == 'gaussian':
        theta = torch.randn((n, 1)) * horizontal_stddev + horizontal_mean
        phi = torch.randn((n, 1)) * vertical_stddev + vertical_mean

    elif mode == 'hybrid':
        if random.random() < 0.5:
            theta = (torch.rand((n, 1)) - 0.5) * 2 * horizontal_stddev * 2 + horizontal_mean
            phi = (torch.rand((n, 1)) - 0.5) * 2 * vertical_stddev * 2 + vertical_mean
        else:
            theta = torch.randn((n, 1)) * horizontal_stddev + horizontal_mean
            phi = torch.randn((n, 1)) * vertical_stddev + vertical_mean

    #elif mode == 'truncated_gaussian':
    #    theta = truncated_normal_(torch.zeros((n, 1))) * horizontal_stddev + horizontal_mean
    #    phi = truncated_normal_(torch.zeros((n, 1))) * vertical_stddev + vertical_mean

    elif mode == 'spherical_uniform':
        theta = (torch.rand((n, 1)) - .5) * 2 * horizontal_stddev + horizontal_mean
        v_stddev, v_mean = vertical_stddev / math.pi, vertical_mean / math.pi
        v = ((torch.rand((n,1) ) - .5) * 2 * v_stddev + v_mean)
        v = torch.clamp(v, 1e-5, 1 - 1e-5)
        phi = torch.arccos(1 - 2 * v)

    else:
        # Just use the mean.
        theta = torch.ones((n, 1), dtype=torch.float) * horizontal_mean
        phi = torch.ones((n, 1),  dtype=torch.float) * vertical_mean

    phi = torch.clamp(phi, 1e-5, math.pi - 1e-5)

    output_points = torch.zeros((n, 3))
    output_points[:, 0:1] = r*torch.sin(phi) * torch.cos(theta)
    output_points[:, 2:3] = r*torch.sin(phi) * torch.sin(theta)
    output_points[:, 1:2] = r*torch.cos(phi)

    return output_points, phi, theta

In [20]:
camera_origin, pitch, yaw = sample_camera_positions(n=points.shape[0])

In [21]:
def create_cam2world_matrix(forward_vector , origin):
  """Takes in the direction the camera is pointing and the camera origin and returns a cam2world matrix."""
  forward_vector = normalize_vecs(forward_vector)
  up_vector = torch.tensor([0, 1, 0], dtype=torch.float).expand_as(forward_vector)

  left_vector = normalize_vecs(torch.cross(up_vector, forward_vector, dim=-1))

  up_vector = normalize_vecs(torch.cross(forward_vector, left_vector, dim=-1))

  rotation_matrix = torch.eye(4).unsqueeze(0).repeat(forward_vector.shape[0], 1, 1)
  rotation_matrix[:, :3, :3] = torch.stack((-left_vector, up_vector, -forward_vector), axis=-1)

  translation_matrix = torch.eye(4).unsqueeze(0).repeat(forward_vector.shape[0], 1, 1)
  translation_matrix[:, :3, 3] = origin

  cam2world = translation_matrix @ rotation_matrix

  return cam2world

In [22]:
forward_vector=normalize_vecs(-camera_origin)
cam2world_matrix=create_cam2world_matrix(forward_vector,camera_origin)

In [23]:
print(cam2world_matrix.shape)
print(cam2world_matrix)

torch.Size([64, 4, 4])
tensor([[[ 8.5673e-01,  1.9992e-01,  4.7543e-01,  4.7543e-01],
         [ 0.0000e+00,  9.2181e-01, -3.8763e-01, -3.8763e-01],
         [-5.1576e-01,  3.3210e-01,  7.8975e-01,  7.8975e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 1.9844e-01, -5.2293e-01,  8.2895e-01,  8.2895e-01],
         [ 0.0000e+00,  8.4578e-01,  5.3354e-01,  5.3354e-01],
         [-9.8011e-01, -1.0588e-01,  1.6784e-01,  1.6784e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        [[ 9.9044e-01, -8.6726e-02, -1.0723e-01, -1.0723e-01],
         [ 0.0000e+00,  7.7754e-01, -6.2884e-01, -6.2884e-01],
         [ 1.3792e-01,  6.2283e-01,  7.7011e-01,  7.7011e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],

        ...,

        [[ 9.8896e-01,  8.4394e-02,  1.2178e-01,  1.2178e-01],
         [ 0.0000e+00,  8.2192e-01, -5.6961e-01, -5.6961e-01],
         [-1.4816e-01,  5.6332e-01,  8.1285e-01,  8.1285e-01],
         [ 0

In [24]:
def transform_sampled_points(points, z_vals, ray_directions, h_stddev=1, v_stddev=1, h_mean=math.pi * 0.5, v_mean=math.pi * 0.5, mode='normal'):
    """Samples a camera position and maps points in camera space to world space."""

    n, num_rays, num_steps, channels = points.shape

    points, z_vals = perturb_points(points, z_vals, ray_directions)


    camera_origin, pitch, yaw = sample_camera_positions(n=points.shape[0], r=1, horizontal_stddev=h_stddev, vertical_stddev=v_stddev, horizontal_mean=h_mean, vertical_mean=v_mean, mode=mode)
    forward_vector = normalize_vecs(-camera_origin)

    cam2world_matrix = create_cam2world_matrix(forward_vector, camera_origin)

    points_homogeneous = torch.ones((points.shape[0], points.shape[1], points.shape[2], points.shape[3] + 1))
    points_homogeneous[:, :, :, :3] = points

    # should be n x 4 x 4 , n x r^2 x num_steps x 4
    transformed_points = torch.bmm(cam2world_matrix, points_homogeneous.reshape(n, -1, 4).permute(0,2,1)).permute(0, 2, 1).reshape(n, num_rays, num_steps, 4)


    transformed_ray_directions = torch.bmm(cam2world_matrix[..., :3, :3], ray_directions.reshape(n, -1, 3).permute(0,2,1)).permute(0, 2, 1).reshape(n, num_rays, 3)

    homogeneous_origins = torch.zeros((n, 4, num_rays))
    homogeneous_origins[:, 3, :] = 1
    transformed_ray_origins = torch.bmm(cam2world_matrix, homogeneous_origins).permute(0, 2, 1).reshape(n, num_rays, 4)[..., :3]

    return transformed_points[..., :3], z_vals, transformed_ray_directions, transformed_ray_origins, pitch, yaw

In [31]:
transformed_points, z_vals, transformed_ray_directions, transformed_ray_origins, pitch, yaw = transform_sampled_points(points,z_vals,rays_d_cam)

ValueError: not enough values to unpack (expected 4, got 3)

In [26]:
transformed_points.shape,z_vals.shape,transformed_ray_directions.shape,transformed_ray_origins.shape,pitch.shape,yaw.shape

NameError: name 'transformed_points' is not defined

In [27]:
transformed_ray_directions_expanded=torch.unsqueeze(transformed_ray_directions,-2).expand(-1,-1,4,-1).reshape(3,8*8*4,3)
transformed_points=transformed_points.reshape(3,8*8*4,3)
print(transformed_ray_directions_expanded.shape,transformed_points.shape)

NameError: name 'transformed_ray_directions' is not defined

In [1]:
import curriculums
from model import siren
curriculum = getattr(curriculums, 'CelebA')
metadata=curriculums.extract_metadata(curriculum,0)
si = getattr(siren, metadata['SIREN'])
si

: 

: 

In [1]:
import curriculums
from model import siren
curriculum = getattr(curriculums, 'CelebA')
metadata=curriculums.extract_metadata(curriculum,0)
metadata

: 

: 

In [39]:
import torch.nn as nn
import torch
class Manifold_predictor(nn.Module):
    """manifold_predictor M
    input (x,y,z) to scalar s

    Args:
        init (str): decision for initial manifold shape
        
    """
    def __init__(self,input_dim=3,hidden_dim=128,
                 act=nn.LeakyReLU(0.2,inplace=True), init='sphere'):
        super().__init__()
        self.init=init
        self.input_dim=input_dim
        self.act=act
        self.hidden_dim=hidden_dim
        self.layer1=nn.Linear(self.input_dim,self.hidden_dim)
        self.layer2=nn.Linear(self.hidden_dim,self.hidden_dim)
        self.layer3=nn.Linear(self.hidden_dim,1)
        # initialize to shpere-like shape
        if self.init=='sphere':
            torch.nn.init.normal_(self.layer1.weight, 0.0, np.sqrt(2)/np.sqrt(hidden_dim))
            torch.nn.init.constant_(self.layer1.bias, 0.0)
            
            torch.nn.init.normal_(self.layer2.weight, 0.0, np.sqrt(2)/np.sqrt(hidden_dim))
            torch.nn.init.constant_(self.layer2.bias,0.0)
                
            torch.nn.init.normal_(self.layer3.weight, mean=2*np.sqrt(np.pi) / np.sqrt(hidden_dim), std=0.000001)
            torch.nn.init.constant_(self.layer3.bias, 0.0)    
    def forward(self,input):
        x=input
        x=self.layer1(x)
        x=self.act(x)
        x=self.layer2(x)
        x=self.act(x)
        return self.layer3(x)

    def calculate_intersection(self,intervals,vals,levels):
        intersections = []
        is_valid = []
        for interval,val,l in zip(intervals,vals,levels):
            x_l = interval[:,:,0]
            x_h = interval[:,:,1]
            s_l = val[:,:,0]
            s_h = val[:,:,1]
            scale = torch.where(torch.abs(s_h-s_l) > 0.05,s_h-s_l,torch.ones_like(s_h)*0.05)
            intersect = torch.where(((s_h-l<=0)*(l-s_l<=0)) & (torch.abs(s_h-s_l) > 0.05),((s_h-l)*x_l + (l-s_l)*x_h)/scale,x_h)
            intersections.append(intersect)
            is_valid.append(((s_h-l<=0)*(l-s_l<=0)).to(intersect.dtype))
        
        return torch.stack(intersections,dim=-2),torch.stack(is_valid,dim=-2) #[batch,N_rays,level,3]    

    def get_intersections(self, input, levels, **kwargs):
        # levels num_l
        batch,N_rays,N_points,_ = input.shape
        
        x = input.reshape(batch,-1,3)

        x=input
        x=self.layer1(x)
        x=self.act(x)
        x=self.layer2(x)
        x=self.act(x)
        s = self.layer3(x)

        s = s.reshape(batch,N_rays,N_points,1)
        s_l = s[:,:,:-1]
        s_h = s[:,:,1:]

        cost = torch.linspace(N_points-1,0,N_points-1).float().to(input.device).reshape(1,1,-1,1)
        x_interval = []
        s_interval = []
        for l in levels:
            r = (s_h-l <= 0) * (l-s_l <= 0) * 2 - 1
            r = r*cost
            _, indices = torch.max(r,dim=-2,keepdim=True)
            x_l_select = torch.gather(input,-2,indices.expand(-1, -1, -1, 3)) # [batch,N_rays,1]
            x_h_select = torch.gather(input,-2,indices.expand(-1, -1, -1, 3)+1) # [batch,N_rays,1]
            s_l_select = torch.gather(s_l,-2,indices)
            s_h_select = torch.gather(s_h,-2,indices)
            x_interval.append(torch.cat([x_l_select,x_h_select],dim=-2))
            s_interval.append(torch.cat([s_l_select,s_h_select],dim=-2))
        
        intersections,is_valid = self.calculate_intersection(x_interval,s_interval,levels)
        
        return intersections,s,is_valid


In [41]:
batch_size=4
img_size=32
fov=12
ray_start=0.88
ray_end=1.10
num_steps=12
points_cam, z_vals, rays_d_cam = get_initial_rays_trig(batch_size, num_steps, resolution=(img_size, img_size), fov=fov, ray_start=ray_start, ray_end=ray_end) # batch_size, pixels, num_steps, 1
transformed_points, z_vals, transformed_ray_directions, transformed_ray_origins, pitch, yaw = transform_sampled_points(points_cam, z_vals, rays_d_cam)
transformed_points_sample = transformed_points.reshape(batch_size, img_size*img_size, -1, 3)
print(transformed_points_sample.shape)
levels = torch.linspace(1, 6, 24)


torch.Size([4, 1024, 12, 3])


In [71]:
manifold=Manifold_predictor()
manifold.get_intersections(transformed_points_sample,levels)[0]

tensor([[[[-0.1321,  0.0175, -0.1301],
          [-0.1321,  0.0175, -0.1301],
          [-0.1321,  0.0175, -0.1301],
          ...,
          [-0.1321,  0.0175, -0.1301],
          [-0.1321,  0.0175, -0.1301],
          [-0.1321,  0.0175, -0.1301]],

         [[-0.1253,  0.0157, -0.1333],
          [-0.1253,  0.0157, -0.1333],
          [-0.1253,  0.0157, -0.1333],
          ...,
          [-0.1253,  0.0157, -0.1333],
          [-0.1253,  0.0157, -0.1333],
          [-0.1253,  0.0157, -0.1333]],

         [[-0.1172,  0.0182, -0.1311],
          [-0.1172,  0.0182, -0.1311],
          [-0.1172,  0.0182, -0.1311],
          ...,
          [-0.1172,  0.0182, -0.1311],
          [-0.1172,  0.0182, -0.1311],
          [-0.1172,  0.0182, -0.1311]],

         ...,

         [[ 0.1010, -0.1412,  0.0067],
          [ 0.1010, -0.1412,  0.0067],
          [ 0.1010, -0.1412,  0.0067],
          ...,
          [ 0.1010, -0.1412,  0.0067],
          [ 0.1010, -0.1412,  0.0067],
          [ 0.1010, -0

In [58]:
manifold(transformed_points_sample).shape

torch.Size([4, 1024, 12, 1])

In [60]:

(-1<=0)*(1<=0)*2-1

-1

In [67]:
(-1<=0)*2-1

1

In [75]:
0.1+1.1== 1.2

False